## TinyLlama Generations


In [1]:
import requests
import json
import os
import random
import time
import uuid
import pandas as pd

OLLAMA_URL = "http://localhost:11434/api/generate"  
MODEL_NAME = "tinyllama"

In [ ]:
# Create test dataset
tinyllama_df = pd.DataFrame(columns=['uuid', 'topic', 'generated_article', 'elapsed_time'])

In [ ]:
topics = ["US - Crime + Justice", "World - Africa", "World - Americas", "World - Asia", "World - Australia",
          "World - China", "World - Europe", "World - India", "World - Middle East", "World - United Kingdom",
          "Politics - CNN Polls", "Politics - Elections", "Business - Tech", "Business - Media", "Business - Markets",
          "Business - Pre-markets", "Business - After-Hours", "Business - Investing", "Business - Markets Now",
          "Health - Fitness", "Health - Food", "Health - Sleep", "Health - Mindfulness", "Health - Relationships",
          "Entertainment - Movies", "Entertainment - Television", "Entertainment - Celebrity", "Tech - Innovate",
          "Tech - Foreseeable Future", "Tech - Innovative Cities", "Style - Arts", "Style - Design", "Style - Fashion",
          "Style - Architecture", "Style - Luxury", "Style - Beauty", "Travel - Destinations", "Travel - Food & Drink",
          "Travel - Lodging and Hotels", "Travel - News", "Sports - Pro Football", "Sports - College Football",
          "Sports - Basketball", "Sports - Baseball", "Sports - Soccer", "Sports - Olympics", "Sports - Hockey",
          "Science - Space", "Science - Life", "Science - Medicine", "Science - Climate", "Science - Solutions",
          "Science - Weather"]

In [ ]:
def generate_article(topics):
    n = len(topics)
    i = random.randint(0, n - 1)
    topic = topics[i]

    prompt = f"""Write a full news article in the style of CNN or DailyMail.
The story should sound realistic, factual, and human-written.
Use natural journalistic language with short and medium-length sentences.
Start with a strong lead paragraph summarizing who, what, where, and when.
Then expand with quotes, context, background, and a final paragraph about next steps or reactions.
Include realistic numbers, dates, and locations.
The topic should be about {topic}
Add 1–3 short quotes attributed to plausible people (officials, witnesses, or experts).
Use neutral tone — no opinions, exaggeration, or bullet points.
Output only the article text (no headline, no lists, no explanation, no "to summarize").
End cleanly after several paragraphs."""

    # Send the request to your local Ollama instance
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": 0.9,
            "top_p": 0.95,
            "top_k": 50
        },
        "num_predict": 750
    }

    response = requests.post(OLLAMA_URL, json=payload, timeout=600)
    response.raise_for_status()
    data = response.json()

    # Return text content + topic for consistency with your Gemma version
    return data.get("response", "").strip(), topic

In [ ]:
# Generate articles
n = 5 # Number of datapoints to create
for i in range(n):
  # Add unique identifier for the row
  tinyllama_df.loc[i, 'uuid'] = str(uuid.uuid4())
  start_time = time.perf_counter()
  response, topic = generate_article(topics)
  end_time = time.perf_counter()
  elapsed_time = end_time - start_time
  tinyllama_df.loc[i, 'topic'] = topic
  tinyllama_df.loc[i, 'generated_article'] = response
  tinyllama_df.loc[i, 'elapsed_time'] = elapsed_time

In [ ]:
tinyllama_df.head()

In [ ]:
for i in range(n):
  print(i, tinyllama_df['generated_article'][i])

In [ ]:
tinyllama_df.to_csv("tinyllama_outputs.csv")
print("CSV saved as 'tinyllama_outputs.csv'")